In [16]:
import numpy as np

In [22]:
import sklearn.naive_bayes as nb

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [3]:
from sklearn.model_selection import GridSearchCV

In [4]:
import pandas as pd

In [12]:
kaggle = pd.read_csv('trocafone_kaggle_test.csv')

In [13]:
labels = pd.read_csv('labels_training_set.csv')

In [10]:
processed = pd.read_csv('features_10.csv')

In [11]:
processed = processed.set_index('person').fillna(0)
processed.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,...,4GB_vio,512MB_compro,512MB_vio,region_sin_unknown,pais_sin_unknown,precio_del_cel_mas_visto,promedio_precio_celulares_vistos,promedio_precio_celulares_comprados,precio_mas_caro_que_vio,precio_mas_barato_que_vio
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,05-17,05-17,6,1,...,False,False,False,Sao Paulo,Brazil,0.0,0.000000,0.0,0.0,0.0
00091926,viewed product,iPhone 6S,Computer,0,34,372,05-03,05-31,448,28,...,False,False,False,Rio Grande do Sul,Brazil,1679.0,1527.091398,0.0,4649.0,100.0
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,03-26,03-26,10,1,...,False,False,False,Minas Gerais,Brazil,1679.0,1342.333333,0.0,1679.0,849.0
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,05-17,05-26,206,10,...,True,False,False,Sao Paulo,Brazil,369.0,479.366013,379.0,2229.0,250.0
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,05-29,05-29,17,1,...,False,False,False,Minas Gerais,Brazil,2229.0,2229.000000,0.0,2229.0,2229.0


In [31]:
notbool = []
for col in processed.columns:
    if processed[col].dtype.name != 'bool':
        notbool.append(col)
processed = processed.drop(notbool,axis=1)        

In [32]:
training_features = labels.set_index('person').join(processed)
predict_features = kaggle.set_index('person').join(processed)
training_labels = training_features['label']
training_features = training_features.drop('label',axis=1)

In [33]:
training_features.head()

,vio_celulares,is_new,busco_productos,clickeo_ad,hizo_checkout,entro_desde_motor,hizo_conversion,filtro_busquedas,busco_marca,visito_sitio,...,16GB_compro,16GB_vio,8GB_compro,8GB_vio,256GB_compro,256GB_vio,4GB_compro,4GB_vio,512MB_compro,512MB_vio
person,,,,,,,,,,,,,,,,,,,,,
0566e9c1,True,True,False,True,True,True,True,True,True,True,...,False,True,True,True,False,False,False,False,False,False
6ec7ee77,False,True,False,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
abe7a2fb,True,True,True,True,True,True,False,True,True,True,...,False,True,False,False,False,False,False,False,False,True
34728364,True,True,False,False,False,True,False,True,True,True,...,False,True,False,False,False,False,False,True,False,False
87ed62de,True,True,False,True,True,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False


## Mean encoding

In [17]:
tg = category_encoders.TargetEncoder(smoothing=True)
encoded_training = tg.fit_transform(np.array(training_features),np.array(training_labels))
encoded_predict = tg.transform(np.array(predict_features))

In [15]:
encoded_training.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.002778,0.042654,0.068258,0,2,0,6,1,14,6.000000,0.066079
1,0.055313,0.050139,0.036301,0,1,3,17,1,2,17.000000,0.042803
2,0.031056,0.033590,0.036301,0,1,4,19,1,0,19.000000,0.042803
3,0.049121,0.030380,0.068258,2,19,189,471,115,9,4.095652,0.042803
4,0.049121,0.060150,0.017361,1,0,2,7,1,9,7.000000,0.017361


In [54]:
params={
    'alpha':[0.0,0.1,0.2,1.0,2.0,10.0,50.3]
}
grid = GridSearchCV(nb.BernoulliNB(),params,scoring='roc_auc',cv=2,verbose=0)

In [64]:
xtr,xte,ytr,yte=train_test_split(training_features,training_labels)

In [65]:
grid.fit(xtr,ytr)

C:\Users\javif\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\javif\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


GridSearchCV(cv=2, error_score='raise',
       estimator=BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [0.0, 0.1, 0.2, 1.0, 2.0, 10.0, 50.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [66]:
grid.score(X=xte,y=yte)

0.7348366744395968